In [3]:
import pandas as pd
import requests
import os

# Features drift

In [4]:
features_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', '03_primary', 'df_agg.csv'))
features_df = pd.read_csv(features_path)

C:\Users\Z478SG\AppData\Local\Temp\ipykernel_8720\693274606.py:2: DtypeWarning: Columns (756,757,761,762,766,767,771,772,776,777,781,782,786,787,791,792) have mixed types. Specify dtype option on import or set low_memory=False.
  features_df = pd.read_csv(features_path)


In [5]:
train_features = features_df[features_df['TARGET'].notna()]
prod_features = features_df[features_df['TARGET'].isna()]

In [6]:
## Faire aussi sur les données pas processées :
# La plupart des .csv n'ont pas le même nombre de lignes que application_train.csv car ils servent à faire une jointure. 
# Dois-je analyser le drift just de application_test par rapport à application_train ? Car si je fais les jointures avec les autres tableau, c'est comme ci je traitais mes données pour en faire des features.
# LA CONSIGNE EST : détecter éventuellement du Data Drift sur les principales features

## OK - Regarder comment fonctionnent les 2 tests statistiques

In [7]:
features = ['PAYMENT_RATE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'DAYS_EMPLOYED', 'OWN_CAR_AGE', 'DAYS_ID_PUBLISH', 'DAYS_EMPLOYED_PERC', 'AMT_CREDIT' , 'CODE_GENDER', 'ANNUITY_INCOME_PERC', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_CONTRACT_TYPE_Cash loans', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_REGISTRATION', 'NAME_FAMILY_STATUS_Married', 'DEF_30_CNT_SOCIAL_CIRCLE', 'REGION_RATING_CLIENT_W_CITY', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'FLAG_DOCUMENT_3', 'INCOME_CREDIT_PERC', 'AMT_REQ_CREDIT_BUREAU_QRT', 'NAME_EDUCATION_TYPE_Secondary / secondary special']

In [8]:
for col in features:
    if train_features[col].dtype == 'bool':
        train_features.loc[:,col] = train_features[col].astype(int)
        prod_features.loc[:,col] = prod_features[col].astype(int)

C:\Users\Z478SG\AppData\Local\Temp\ipykernel_8720\441531714.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 1 0 ... 1 0 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  train_features.loc[:,col] = train_features[col].astype(int)
C:\Users\Z478SG\AppData\Local\Temp\ipykernel_8720\441531714.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 0 1 ... 0 1 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  prod_features.loc[:,col] = prod_features[col].astype(int)
C:\Users\Z478SG\AppData\Local\Temp\ipykernel_8720\441531714.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 0 ... 1 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  train_fea

In [9]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# Define column mapping if necessary
column_mapping = ColumnMapping(
    # target='TARGET',  # Replace with your target column name if applicable
    # prediction='PREDICTION',  # Replace with your prediction column name if applicable
    numerical_features=features,  # Replace with your numerical feature columns
    categorical_features=[]  # Replace with your categorical feature columns
)

# Create the data drift report
report = Report(metrics=[
    DataDriftPreset(),
])

report.run(reference_data=train_features[features], current_data=prod_features[features], column_mapping=column_mapping)

# Save the report as an HTML file
features_drift_report_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', '08_reporting', 'features_drift_report.html'))
report.save_html(features_drift_report_path)


La distance de Wasserstein mesure le coût minimal de transformation d'une distribution en une autre, calculant essentiellement la quantité de "travail" nécessaire pour reshaper une distribution. Elle est particulièrement sensible aux changements de forme et de localisation des données, offrant une mesure intuitive de la dissimilarité entre deux ensembles de valeurs.

La distance de Jensen-Shannon, quant à elle, évalue la similarité globale entre deux distributions de probabilité en calculant une moyenne symétrique des divergences. Toujours comprise entre 0 et 1, elle fournit une mesure robuste des différences probabilistes, permettant de quantifier à quel point deux distributions s'écartent l'une de l'autre de manière globale et symétrique.

In [10]:
import webbrowser

# Open the HTML report in the default web browser
webbrowser.open(report_path)


True

# Raw data drift

In [11]:
train_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', '01_raw', 'application_train.csv'))
prod_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', '01_raw', 'application_test.csv'))

In [19]:
train_data = pd.read_csv(train_data_path).drop('TARGET', axis=1)
prod_data = pd.read_csv(prod_data_path)

In [20]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# Define column mapping if necessary
column_mapping = ColumnMapping(
    # target='TARGET',  # Replace with your target column name if applicable
    # prediction='PREDICTION',  # Replace with your prediction column name if applicable
    numerical_features=train_data.columns.tolist(),  # Replace with your numerical feature columns
    categorical_features=[]  # Replace with your categorical feature columns
)

# Create the data drift report
report = Report(metrics=[
    DataDriftPreset(),
])

report.run(reference_data=train_data, current_data=prod_data, column_mapping=column_mapping)

# Save the report as an HTML file
raw_data_drift_report_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', '08_reporting', 'raw_data_drift_report.html'))
report.save_html(raw_data_drift_report_path)


In [21]:
import webbrowser

# Open the HTML report in the default web browser
webbrowser.open(report_path)

True